In [1]:
from url import *
from rapidfuzz import process

In [2]:
# url = "https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/"
url = "https://www.allrecipes.com/one-pot-chicken-pomodoro-recipe-8730087/"
# fetch_page_from_url(url)

### Test extract methods per step below ###

ingredients, instructions = fetch_page_from_url(url)

print(ingredients)
print(instructions)

[{'quantity': '1', 'unit': 'pound', 'name': ' skinless boneless chicken breasts', 'preparation': 'thinly sliced'}, {'quantity': '1', 'unit': 'teaspoon', 'name': ' salt'}, {'quantity': '1', 'unit': 'teaspoon', 'name': ' granules', 'descriptor': 'garlic'}, {'quantity': '1', 'unit': 'teaspoon', 'name': ' seasoning', 'descriptor': 'Italian'}, {'quantity': '2', 'unit': 'tablespoons', 'name': ' oil', 'preparation': 'divided', 'descriptor': 'olive'}, {'quantity': '1', 'name': ' shallot', 'preparation': 'minced'}, {'quantity': '4', 'unit': 'cloves', 'name': ' garlic', 'preparation': 'minced'}, {'quantity': '1', 'unit': '(28 ounce) can', 'name': ' tomatoes', 'preparation': 'crushed'}, {'quantity': '1/2', 'unit': 'teaspoon', 'name': ' salt'}, {'quantity': '1', 'unit': 'teaspoon', 'name': ' white sugar'}, {'quantity': '1/2', 'unit': 'teaspoon', 'name': ' pepper flakes', 'preparation': 'crushed', 'descriptor': 'red'}, {'quantity': '1', 'unit': 'tablespoon', 'name': ' basil', 'preparation': 'choppe

In [3]:
alternatives = find_ingredients(ingredients, to_vegetarian)

print(alternatives)

{'skinless boneless chicken breasts': ['tofu', 'seitan'], 'chicken broth': ['vegetable broth', 'mushroom broth']}


In [14]:
for i in instructions:
    print(i)
    step_ingredients = get_step_information(i, ingredients, "ingredients", threshold=60)
    for s in step_ingredients:
        # print(s.strip(), alternatives[0].keys(), str(s).strip() in list(alternatives[0].keys()))
        if s.strip() in alternatives.keys():
            print(' or '.join(alternatives[s.strip()]))
            print("\n")

Gather all ingredients
Heat a deep cast iron skillet over medium heat.
Season chicken breasts with salt, garlic, and Italian seasoning.
tofu or seitan


Pour 2 teaspoons oil into the skillet
tofu or seitan


Once hot, add chicken
tofu or seitan


Cook until golden, turning once, about 4 minutes per side
Move chicken to one side of the skillet.
tofu or seitan


Pour remaining oil into the skillet
tofu or seitan


Add minced shallot; cook until softened, about 3 minutes
Add the garlic, stir until fragrant, about 1 minute.
Add the crushed tomatoes, salt, sugar, red pepper flakes, and basil
Pour in chicken broth and wine
tofu or seitan


vegetable broth or mushroom broth


Stir to combine.
Add pasta to the skillet
tofu or seitan


Nestle chicken pieces on top of pasta
tofu or seitan


Reduce heat to medium-low
tofu or seitan


Cover, and cook until pasta is tender with a bite, about 8 minutes
Taste and adjust seasoning; serve immediately.


In [5]:
"skinless boneless chicken breasts" in alternatives.keys()

True

In [6]:
instructions

['Gather all ingredients',
 'Heat a deep cast iron skillet over medium heat.',
 'Season chicken breasts with salt, garlic, and Italian seasoning.',
 'Pour 2 teaspoons oil into the skillet',
 'Once hot, add chicken',
 'Cook until golden, turning once, about 4 minutes per side',
 'Move chicken to one side of the skillet.',
 'Pour remaining oil into the skillet',
 'Add minced shallot; cook until softened, about 3 minutes',
 'Add the garlic, stir until fragrant, about 1 minute.',
 'Add the crushed tomatoes, salt, sugar, red pepper flakes, and basil',
 'Pour in chicken broth and wine',
 'Stir to combine.',
 'Add pasta to the skillet',
 'Nestle chicken pieces on top of pasta',
 'Reduce heat to medium-low',
 'Cover, and cook until pasta is tender with a bite, about 8 minutes',
 'Taste and adjust seasoning; serve immediately.']

In [7]:
modify_recipe_instructions(instructions, ingredients, alternatives)

['Gather all ingredients',
 'Heat a deep cast iron skillet over medium heat.',
 'Tofu or seitan with salt, garlic, and italian seasoning.',
 'Pour 2 teaspoons oil into the skillet',
 'Once hot, add tofu or seitan',
 'Cook until golden, turning once, about 4 minutes per side',
 'Move chicken to one side of the skillet.',
 'Pour remaining oil into the skillet',
 'Add minced shallot; cook until softened, about 3 minutes',
 'Add the garlic, stir until fragrant, about 1 minute.',
 'Add the crushed tomatoes, salt, sugar, red pepper flakes, and basil',
 'Pour in vegetable broth or mushroom broth and wine',
 'Stir to combine.',
 'Add pasta to the skillet',
 'Nestle chicken pieces on top of pasta',
 'Reduce heat to medium-low',
 'Cover, and cook until pasta is tender with a bite, about 8 minutes',
 'Taste and adjust seasoning; serve immediately.']

In [ ]:
instructions = modify_recipe_instructions(instructions, alternatives)

print(instructions)

TypeError: modify_recipe_instructions() missing 1 required positional argument: 'alternatives'